# Convolutional Neural Network for Small Data Sets

การทดลองสร้าง convolutional neural network สำหรับจำแนกประเภทรูปภาพเป็นสองคลาส (สุนัข และ แมว)

ข้อมูลแบ่งเป็นสามส่วนคือ training dataset, validation dataset, test dataset

* **training dataset**. เป็นชุดข้อมูลตัวอย่างสำหรับใช้เทรนโมเดลเพื่อหาค่าโมเดลพารามิเตอร์ที่เหมาะสม
* **validation dataset**. เป็นชุดข้อมูลสำหรับเลือกค่า hyperparameters (คือพารามิเตอร์ที่ไม่ได้มาจากการเทรน เช่น จำนวน hidden layers ที่เหมาะสม, ค่า learning rate, ...) 
* **test dataset**. เป็นชุดข้อมูลสำหรับใช้เปรียบเทียบประสิทธิภาพ (unbiased evaluation) ของโมเดลที่ได้จากการเทรน 

### Load data from a file system

#### Set directory paths

In [ ]:
import os
train_dir = 'data/cats_and_dogs_small/train'
validation_dir = 'data/cats_and_dogs_small/validation'
test_dir = 'data/cats_and_dogs_small/test'

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [ ]:
print('total train cat images:', len(os.listdir(train_cats_dir)))
print('total train dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

จากผลที่แสดงใน cell ก่อนหน้า 
* training set ประกอบด้วย รูปภาพแมว 1000 รูป และ รูปภาพสุนัข 1000 รูป 
* validation set ประกอบด้วย รูปภาพแมว 500 รูป และ รูปภาพสุนัข 500 รูป
* test set ประกอบด้วย รูปภาพแมว 500 รูป และ รูปภาพสุนัข 500 รูป

#### Load and Pre-process Image Data Set

เนื่องจากข้อมูลรูปภาพมีขนาดไม่เท่ากัน ก่อนป้อนข้อมูลให้กับนิวรอนเน็ตเวิร์ก เราจำเป็นต้องแปลงข้อมูลรูปภาพในชุดข้อมูลให้อยู่ในรูปแบบและสเกลที่เหมาะสม ตรงกันทั้ง training set, validation set, และ test set ก่อน

โดยจะใช้คลาส `ImageDataGenerator` ในแพ็คเกจ `keras.preprocessing.image` สำหรับ pre-process ข้อมูลด้วย operations ดังต่อไปนี้ คือ
* **Rescale**. แปลงช่วงของค่าของข้อมูลให้อยู่ในช่วง 0.0 ถึง 1.0 โดยการหารค่าของข้อมูลทุกตัวด้วยค่าสูงสุดที่เป็นไปได้ (สำหรับข้อมูลรูปภาพ ค่าในแต่ละพิกเซล จะมีค่าตั้งแต่ 0 ถึง 255)
* **Resize**. แปลงขนาดของรูปภาพให้มีขนาด 150x150 pixels.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator object, set rescaler with the value 1.0/255 (normalize to 0-1)
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a Generator object for training dataset
train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=(150, 150),
                batch_size=20,
                class_mode='binary')

# Create a Generator object for validation dataset
validation_generator = validation_datagen.flow_from_directory(
                validation_dir,
                target_size=(150, 150),
                batch_size=20,
                class_mode='binary')

# Create a Generator object for test dataset
test_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(150, 150),
                batch_size=20,
                class_mode='binary')

In [ ]:
# check shape of images in the training set
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

### Build model

กำหนดโครงสร้างของ convnet โดยโมเดลของเราจะประกอบด้วยเลเยอร์ต่างๆ ดังนี้

1. Convolutional Layer ลำดับที่ 1. ประกอบด้วย ReLU filters ขนาด 3x3 จำนวน 32 ฟิลเตอร์, อินพุทของเลเยอร์นี้คือ ข้อมูลรูปภาพสีขนาด 150x150x3, เอาท์พุทมีมิติ 148x148x32
2. Max Pooling Layer ลำดับที่ 1. ประกอบด้วย ReLU filters ขนาด 2x2 จำนวน 32 ฟิลเตอร์, เอาท์พุทมีมิติ 74x74x32
3. Convolutional Layer ลำดับที่ 2. ประกอบด้วย filters ขนาด 3x3x32 จำนวน 64 ฟิลเตอร์, เอาท์พุทมีมิติ 72x72x64
4. Max Pooling Layer ลำดับที่ 2. ประกอบด้วย filters ขนาด 2x2 จำนวน 64 ฟิลเตอร์, เอาท์พุทมีมิติ 36x36x64
5. Convolutional Layer ลำดับที่ 3. ประกอบด้วย filters ขนาด 3x3x64 จำนวน 128 ฟิลเตอร์, เอาท์พุทมีมิติ 34x34x128
6. Max Pooling Layer ลำดับที่ 3. ประกอบด้วย filters ขนาด 2x2 จำนวน 128 ฟิลเตอร์, เอาท์พุทมีมิติ 17x17x128
7. Convolutional Layer ลำดับที่ 4. ประกอบด้วย filters ขนาด 3x3x128 จำนวน 128 ฟิลเตอร์, เอาท์พุทมีมิติ 15x15x128
8. Max Pooling Layer ลำดับที่ 4. ประกอบด้วย filters ขนาด 2x2 จำนวน 128 ฟิลเตอร์, เอาท์พุทมีมิติ 7x7x128
9. Flatten Layer. แปลงข้อมูลอินพุทขนาด 7x7x128 เป็น 6272x1
10. Dense Layer ลำดับที่ 1. นิวรอนชนิด ReLU จำนวน 512 เชื่อมต่อกับเลเยอร์ก่อนหน้าและเลเยอร์ถัดไปแบบ fully connected
11. Output Layer. ประกอบด้วยนิวรอนขนิด sigmoid จำนวน 1 นิวรอน

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(128,(3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(128,(3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

### Train Model

เทรนโมเดล โดยกำหนด loss function เป็น binary crossentropy, ใช้ optimizer เป็น RMSprop สำหรับวิธีการวัดประสิทธิภาพโมเดลใช้ accuracy.

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
            train_generator, steps_per_epoch=100, 
            epochs=100,
            validation_data=validation_generator,
            validation_steps=50)

#### Save Model for Later use

In [ ]:
model.save('cats_and_dogs_small_1.h5')

#### แสดงกราฟ Training Accuracy และ Validation Accuracy ในแต่ละรอบของการเทรนโมเดล

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()